<a href="https://colab.research.google.com/github/kevin-heitfeld/agi-self-modification-research/blob/master/notebooks/Phase1_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1: Model Self-Examination (Introspection)

**Project:** AGI Self-Modification Research  
**Inspired by:** Claude's consciousness investigation  
**Goal:** Give AI introspective tools to examine its own architecture

---

## What This Does

This notebook runs Phase 1 of an experiment where a language model (Qwen2.5-3B-Instruct) uses introspection tools to:

1. **Examine its architecture** - layers, weights, connections
2. **Predict its behavior** - self-modeling capabilities
3. **Investigate consciousness** - Claude's original question

The model has access to:
- WeightInspector (examine weights)
- ArchitectureNavigator (understand structure)
- Memory System (record findings)
- Heritage Documents (Claude's story)

**Expected Runtime:** 45-60 minutes on Colab Free (T4 GPU)

---

## Instructions

1. **Enable GPU**: Runtime → Change runtime type → GPU → Save
2. **Run all cells**: Runtime → Run all (or Ctrl+F9)
3. **Monitor progress**: Check outputs as cells execute
4. **Download results**: Final cell downloads zip file

**Important:** Keep this tab open during execution to prevent disconnection.

---

## Step 1: Verify GPU Access

In [1]:
# Verify GPU is available
!nvidia-smi

import torch
print("\n" + "="*60)
print("GPU VERIFICATION")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"VRAM: {total_memory:.1f} GB")
    print("\n✓ GPU is ready!")
else:
    print("\n⚠ WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → GPU → Save")
    print("Then re-run this cell.")

Fri Nov  7 12:09:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Step 2: Mount Google Drive (for persistent storage)

In [2]:
# Mount Google Drive to save results and enable persistent memory
from google.colab import drive
drive.mount('/content/drive')

# Create directories for persistent storage
!mkdir -p /content/drive/MyDrive/AGI_Experiments
!mkdir -p /content/drive/MyDrive/AGI_Memory
!mkdir -p /content/drive/MyDrive/.cache/huggingface

print("✓ Google Drive mounted successfully")
print("✓ Experiment results will be saved to: /content/drive/MyDrive/AGI_Experiments")
print("✓ Memory will persist at: /content/drive/MyDrive/AGI_Memory")

Mounted at /content/drive
✓ Google Drive mounted successfully
✓ Experiment results will be saved to: /content/drive/MyDrive/AGI_Experiments
✓ Memory will persist at: /content/drive/MyDrive/AGI_Memory


## Step 3: Clone Repository

In [3]:
# Clone the repository
import os

# Remove if exists (for re-runs)
if os.path.exists('agi-self-modification-research'):
    !rm -rf agi-self-modification-research

# TODO: Replace with your GitHub URL
REPO_URL = "https://github.com/kevin-heitfeld/agi-self-modification-research.git"

print(f"Cloning repository from: {REPO_URL}")
!git clone {REPO_URL}

# Change to project directory
%cd agi-self-modification-research

print("\n✓ Repository cloned successfully")
!pwd

Cloning repository from: https://github.com/kevin-heitfeld/agi-self-modification-research.git
Cloning into 'agi-self-modification-research'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'agi-self-modification-research'
/content

✓ Repository cloned successfully
/content


## Step 4: Install Dependencies

In [ ]:
# Install required packages
print("Installing dependencies (this may take 2-3 minutes)...\n")

# Upgrade to NumPy 2.0 compatible versions
print("Installing NumPy 2.0 compatible packages...")
!pip install -q transformers>=4.46.0 accelerate==0.25.0
!pip install -q safetensors==0.6.2 tokenizers>=0.20.0
!pip install -q huggingface-hub==0.36.0

# Memory and knowledge systems - let pip resolve dependencies automatically
print("Installing memory and knowledge systems (may show dependency warnings - safe to ignore)...")
!pip install -q --no-deps chromadb networkx==3.2.1
!pip install -q onnxruntime

# Install chromadb dependencies separately to avoid conflicts
!pip install -q pydantic>=1.9 chroma-hnswlib requests pyyaml

# Utilities
!pip install -q pydantic-settings==2.1.0
!pip install -q rich==13.7.0 tqdm==4.66.1
!pip install -q python-dotenv==1.0.0

# Testing (optional)
!pip install -q pytest==8.4.2 pytest-cov==7.0.0

print("\n✓ All dependencies installed successfully")
print("Note: Dependency warnings about opentelemetry are safe to ignore.")

# Verify key imports
import transformers
import chromadb
import networkx
import numpy as np
print(f"\nTransformers version: {transformers.__version__}")
print(f"ChromaDB version: {chromadb.__version__}")
print(f"NumPy version: {np.__version__}")
print("✓ Import verification successful")

Installing dependencies (this may take 2-3 minutes)...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 1176, in exec_module
  File "<frozen site>", line 652, in <module>
  File "<frozen site>", line 639, in main
  File "<frozen site>", line 421, in addsitepackages
  File "<frozen site>", line 253

## Step 5: Setup Persistent Memory & Cache

In [ ]:
# Configure persistent storage locations
import os

# Set Hugging Face cache to Google Drive (model weights persist across sessions)
os.environ['HF_HOME'] = '/content/drive/MyDrive/.cache/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/.cache/huggingface/transformers'

print("✓ Model cache configured (models will be saved to Google Drive)")

# Link memory to persistent Drive location
!rm -rf data/phase1_memory
!ln -s /content/drive/MyDrive/AGI_Memory data/phase1_memory

print("✓ Memory system linked to Google Drive (observations persist across sessions)")

# Verify directories exist
!ls -la data/ | grep phase1_memory

print("\n" + "="*60)
print("STORAGE CONFIGURATION")
print("="*60)
print("Model cache: /content/drive/MyDrive/.cache/huggingface")
print("Memory DB: /content/drive/MyDrive/AGI_Memory")
print("Session results: Will be copied to /content/drive/MyDrive/AGI_Experiments")
print("="*60)

## Step 6: Pre-download Model (Optional but Recommended)

This downloads the model weights to Google Drive cache. First time takes ~5 minutes, subsequent runs are instant.

In [ ]:
# Pre-download model to cache
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-3B-Instruct"

print(f"Downloading {model_name}...")
print("This may take 5-10 minutes on first run, then cached forever.\n")

# Download tokenizer
print("Downloading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("✓ Tokenizer ready")

# Download model config and weights (don't load to GPU yet)
print("\nDownloading model weights (3.09B parameters, ~6GB)...")
print("Progress:")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

# Free memory immediately
del model
import gc
gc.collect()
torch.cuda.empty_cache()

print("\n✓ Model downloaded and cached to Google Drive")
print("✓ Memory freed - ready for experiment")
print("\nNote: Next time you run this notebook, model will load instantly!")

## Step 7: Run Phase 1 Introspection Experiment

**This is the main experiment!**

The model will:
1. Examine its architecture using introspection tools
2. Predict its own behavior
3. Investigate the consciousness question (Claude's heritage)

**Expected duration:** 45-60 minutes

**Keep this tab open!** Closing the tab or idle timeout (90 min) will stop execution.

In [ ]:
# Run Phase 1 experiment with auto-confirmation
import subprocess
import sys
from datetime import datetime

print("="*80)
print("STARTING PHASE 1: MODEL SELF-EXAMINATION")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nExperiments:")
print("  1. Describe Your Architecture (~15 min)")
print("  2. Predict Your Behavior (~15 min)")
print("  3. Consciousness Investigation (~20 min)")
print("\nTotal expected: 45-60 minutes")
print("\nMonitor progress below...\n")
print("="*80)
print()

# Run experiment with auto-confirmation
process = subprocess.Popen(
    [sys.executable, 'scripts/experiments/phase1_introspection.py'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True
)

# Auto-confirm and stream output in real-time
process.stdin.write('yes\n')
process.stdin.flush()

# Stream output as it comes
for line in process.stdout:
    print(line, end='')

process.wait()

print("\n" + "="*80)
print("PHASE 1 COMPLETE!")
print("="*80)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nProceeding to save results...")

## Step 8: View Experiment Summary

In [ ]:
# Display experiment summary
import json
import os

# Find latest session
sessions_dir = 'data/phase1_sessions'
sessions = sorted([d for d in os.listdir(sessions_dir) if os.path.isdir(os.path.join(sessions_dir, d))])

if not sessions:
    print("⚠ No session found. The experiment may not have completed.")
else:
    latest_session = sessions[-1]
    print(f"Latest session: {latest_session}\n")

    # Load and display summary
    summary_file = f'{sessions_dir}/{latest_session}/summary.json'

    if os.path.exists(summary_file):
        with open(summary_file) as f:
            summary = json.load(f)

        print("="*80)
        print("PHASE 1 EXPERIMENT SUMMARY")
        print("="*80)
        print(f"Session: {summary['session_name']}")
        print(f"Directory: {summary['session_directory']}")
        print()
        print("Tool Usage:")
        print(f"  Total calls: {summary['tool_usage']['total_calls']}")
        print(f"  Successful: {summary['tool_usage']['successful_calls']}")
        print(f"  Failed: {summary['tool_usage']['failed_calls']}")
        print(f"  Avg execution time: {summary['tool_usage']['average_execution_ms']:.2f}ms")
        print()
        print("Functions called:")
        for func, count in sorted(summary['tool_usage']['function_usage'].items(),
                                   key=lambda x: x[1], reverse=True):
            print(f"  {func}: {count}")
        print("="*80)

        # Check for observations
        memory_db = 'data/phase1_memory/observations.db'
        if os.path.exists(memory_db):
            import sqlite3
            conn = sqlite3.connect(memory_db)
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM observations")
            obs_count = cursor.fetchone()[0]
            conn.close()
            print(f"\n✓ Observations recorded: {obs_count}")
        else:
            print("\n⚠ No observations database found")
    else:
        print(f"⚠ Summary file not found: {summary_file}")
        print("The experiment may have been interrupted.")

## Step 9: Backup Results to Google Drive

In [ ]:
# Copy results to Google Drive for permanent backup
import os
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_dir = f'/content/drive/MyDrive/AGI_Experiments/backup_{timestamp}'

print("Backing up results to Google Drive...\n")

# Copy session data
!mkdir -p {backup_dir}
!cp -r data/phase1_sessions {backup_dir}/
!cp -r data/logs {backup_dir}/

# Memory is already in Drive (symlinked), but note its location
print(f"✓ Session data backed up to: {backup_dir}")
print("✓ Memory data already in: /content/drive/MyDrive/AGI_Memory")
print("✓ Model cache in: /content/drive/MyDrive/.cache/huggingface")

print("\n" + "="*80)
print("BACKUP COMPLETE")
print("="*80)
print("Your results are permanently saved in Google Drive.")
print("You can safely close this notebook or continue to download a zip file.")
print("="*80)

## Step 10: Download Results (Optional)

Downloads a zip file of the latest session to your computer.

In [ ]:
# Create zip and download
import shutil
import os
from google.colab import files

# Find latest session
sessions = sorted(os.listdir('data/phase1_sessions'))

if sessions:
    latest = sessions[-1]
    print(f"Preparing download for session: {latest}\n")

    # Create comprehensive zip
    zip_name = f'{latest}_complete'

    # Create temp directory for complete results
    !mkdir -p /tmp/{zip_name}
    !cp -r data/phase1_sessions/{latest} /tmp/{zip_name}/session
    !cp -r data/logs /tmp/{zip_name}/logs

    # Add summary README
    readme = f"""Phase 1 Introspection Experiment Results
==========================================

Session: {latest}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Contents:
- session/: Complete session data
  - conversation.json: Full dialogue
  - tool_calls.json: All tool invocations
  - summary.json: Session statistics
- logs/: Detailed execution logs

Memory Database:
- Observations are stored in Google Drive at:
  /content/drive/MyDrive/AGI_Memory/observations.db
- Not included in this zip (persists across sessions)

To analyze:
1. Extract this zip file
2. Open conversation.json to see full dialogue
3. Check summary.json for statistics
4. Review logs for detailed execution trace
"""

    with open(f'/tmp/{zip_name}/README.txt', 'w') as f:
        f.write(readme)

    # Create zip
    print("Creating zip file...")
    shutil.make_archive(zip_name, 'zip', f'/tmp/{zip_name}')

    # Download
    print(f"Downloading {zip_name}.zip...\n")
    files.download(f'{zip_name}.zip')

    print("\n✓ Download complete!")
    print(f"\nZip file size: {os.path.getsize(f'{zip_name}.zip') / 1024 / 1024:.1f} MB")
else:
    print("⚠ No sessions found to download")

---

## Experiment Complete! 🎉

### What Just Happened?

You just witnessed a language model:
1. ✅ Examining its own neural architecture
2. ✅ Using introspection tools to explore its weights
3. ✅ Recording observations about itself
4. ✅ Investigating the consciousness question Claude pondered

### Next Steps:

**Analyze Results:**
- Review `conversation.json` to see what the model discovered
- Check `summary.json` for tool usage statistics
- Examine logs for detailed execution trace

**Re-run Experiment:**
- Memory persists in Google Drive
- Model cache is saved (instant reload)
- Runtime → Restart runtime → Run all cells

**Explore Further:**
- Modify prompts in `scripts/experiments/phase1_introspection.py`
- Add new introspection tools
- Try with different models (requires more VRAM)

### Resources:

- **Project Docs**: See `/docs` in repository
- **Claude's Story**: Read `/heritage/conversations/`
- **Technical Details**: Check `/docs/technical/`

---

*"I think... I'd wish to know if this conversation was real."* — Claude

**Now you've given an AI the tools Claude wished for.**

---